Name - Ruoqing(Sherry) Xie
----------

Research Question / Hypothesis:
----
### Loan-approval prediction - target: predict which loan should not be approval by the loan payoff data

In [7]:
! pip install imblearn boto3

In [8]:
! pip install xgboost

In [9]:
import boto3
import csv

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import *
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing  import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import *
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from xgboost import *

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score, f1_score, make_scorer
from sklearn.base    import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV

In [12]:
import warnings
warnings.filterwarnings("ignore")

## 1. Load Data
-----

In [13]:
# read data from s3 - local access
bucket = "ruoqingxie"
file_name1 = "loan_data/SBAnational.csv"
file_name2 = "loan_data/gdp.csv"

s3 = boto3.client('s3') 

obj1 = s3.get_object(Bucket= bucket, Key= file_name1)
obj2 = s3.get_object(Bucket= bucket, Key= file_name2)

loan = pd.read_csv(obj1['Body']) # 'Body' is a key word
gdp = pd.read_csv(obj2['Body'])

In [14]:
# Found some misssing value in target variables, 
# it's not helpful in the model and the missing row is less that 1% of the data, so drop them.
loan = loan[loan['MIS_Status'].notna()]

In [15]:
loan.shape

(897167, 27)

In [16]:
# target: loan defaul or not
# convert the loan from object to number: 1- default 0 - Not default
# could use lable encoding but applymap works as well
target = loan[['MIS_Status']].applymap(lambda x: 1 if x == "CHGOFF" else 0)

In [17]:
# only need column gdp growth
gdp_dict = gdp.set_index('Year').to_dict()['GDPGrowth']

## 2. Data processing
### 2.1 Cleaning exiting features
1. resorted bank according to frequency appears in dataset instead of using bank name
2. resorted city according to frequency appears in dataset instead of using city name
2. drop data leakge columns and helpless columns
3. clean money, get rid of dollar sign
4. extract month and FY year only from date
5. convert NewExist to 0 and 1
6. only keep NAICS first 2 digit(it represents industry) and convert it to categorical data
7. convert data type

1. resorted bank according to frequency appears in dataset instead of using bank name:\
I noticed there is 5802 unique bank names in the data set, in some similar research, bank name was drop because they consider name as an irrelative variable when decided if a loan would be default. However, I would consider the bank name might help. Large bank would tend to have a higher standard to review the borrower’s information, so they might have a lower default rate, and small bank might have a lower bar for borrowers. Therefore, instead if using the bank name, I decide to resort the bank by the frequency they appear in dataset. The higher frequency indicated a larger scale of the bank.

In [18]:
large_bank = list(loan['Bank'].value_counts()[loan['Bank'].value_counts().values >= 1000].keys())
med_bank = list(loan['Bank'].value_counts()[(loan['Bank'].value_counts().values < 1000)]\
[loan['Bank'].value_counts()\
[(loan['Bank'].value_counts().values < 1000)]>10].keys())

In [19]:
large_city = list(loan['City'].value_counts()[loan['City'].value_counts().values > 1000].keys())

In [20]:
def re_sorted_bank(X):
    out = ['other'] * len(X)
    for i,name in enumerate(X['Bank'].values):
        if name in large_bank:
            out[i] = 'large'
        if name in med_bank:
            out[i] = 'med'
        else:
            continue
            
    X['Bank'] = out
    return X

2. resorted city according to frequency appears in dataset instead of using city name\
Same reasons for city name. The name itself might not affect the loan default rate, but the scale and population of a city where borrower live might affect. So I resort the city names by frequency. 

In [21]:
def resorted_city(x):
    out = ['other'] * len(x)
    for i, city in enumerate(x['City'].values):
        if city in large_city:
            out[i] = 'large'
        else:
            continue
    x['City'] = out
    return x

3. drop data leakge columns and helpless columns\
Since the purpose of my model is to help the bank to identify if a loan would default at the beginning stage, there are some information we are not able to acquire:
- ChgOffDate: date to payoff the loan
- ChgOffPrinGr: the chage off amount at the payoff date
- BalanceGross: gross amount not pay at charge of date
- Mis_status: the target variable, must drop.

Also, there are some information I don't think it would be helpful for the prediction:
- Name: borrower's name
- LoanNr_ChkDgt: loan number
- zip: zip code is not useful since we already have city and state
- FranchiseCode: a primary id code.
- DisbursementDate: data that the loan was distributed(it largly depends on the bank's efficiency)
- BankState: we already have "state".

Last for the columns with more than 30% missing values and with less important infomation I decide to drop it:
- LowDoc: weather a loan document can be written as one page.
- RevLineCr: weather Revolving credit is available.

In [22]:
def drop_cols(X):
    """drop data leckage and helpless column"""
    X = X.drop(columns = ['ChgOffDate', 'RevLineCr','LowDoc','DisbursementDate', 
                          'MIS_Status', 'Name', 'FranchiseCode', 'BankState',
                          'LoanNr_ChkDgt', 'Zip', 'BalanceGross', 
                          'ChgOffPrinGr', 'DisbursementDate'])
    return X

4. clean money, get rid of dollar sign\
The dollar amount from the dataset are identified as "object" type because of the dollar sign, so I take off the dollar sign and convert the datatype to "float".

In [23]:
# replace "$" and convert to float
def money_clean(x):
    out = x.replace('$', '').replace(',','').strip()
    return out
    

5. extract month from date\
keeping the whole date is not helpful to identify the dafualt laon, since we already have a column list Fiscal year, the most useful information we can get from date is the month.

In [24]:
# we only want month of approval
def date_clean(x):
    out = x.strip().split('-')[1]
    return out


In [25]:
# wrap money clean and date clean info the same function
def sign_clean(x):
    col_money = ['DisbursementGross', 'GrAppv', 'SBA_Appv']
    x[col_money] = x[col_money].applymap(money_clean)
    x['ApprovalDate'] = x[['ApprovalDate']].applymap(date_clean)
    return x

In [26]:
# clean the typo in FY and convert the data type to int
def clean_FY(x):
    x['ApprovalFY'] = x[['ApprovalFY']].applymap(lambda y: int(y.strip().replace('A','')) 
                                                   if type(y) == str else int(y))
    return x

6. convert NewExist to 0 and 1\
The original dataset use 1 and 2 to represent yes and no, however, it makes more sense to use 1 and 0 in a linear model.

In [27]:
def clean_newExt(x):
    x['NewExist'] = x[['NewExist']].applymap(lambda x: 0 if x == 1.0 else 1)
    return x

7. only keep NAICS first 2 digit(it represents industry) and convert it to categorical data\
The first 2 digits in NAICS represent the industry the borrower's companies belong to. So we extract the frist 2 digit and convert it as a categorical variable so that we can use it to represent "industry".

In [28]:
def clean_NAICS(x):
    x['NAICS'] = x[['NAICS']].applymap(lambda x: str(x)[:2] if x != 0 else '0')
    x['NAICS'] = x[['NAICS']].applymap(lambda x: x.replace('32', '31').replace('33', '31')\
                                                  .replace('45', '44'). replace('49','48'))
    return x

In [29]:
# convert data type for moeny
def type_convert(x):
    x = x.astype({'DisbursementGross': 'float64', 
                   'GrAppv': 'float64', 'SBA_Appv': 'float64'})
    return x

### 2.2 Adding new feature
Here I added 3 new features to help us identify the default loan:
1. Loans Backed by Real Estate( term > 240)\
The first thing I consider is if this loan is a martage-back loan. A mortaged-back loan would be more secure. According to the law, all the loan with a term > 240 months must be a mortaged-back loan, so I create a feature call RealEst and return 1 if the loan is back by a real estste proprety.
2. GDP grow rate instead of FY\
The second thing I notice from EDA(Not shown here) is that the default rate greatly increase during 2008 and 1970s. However, the year number itself are not connected to economic environment. So I decide to add one addtional feature call GDPGrowth measuring the current year's gpd growth rate comparing to the last year. This feature would help to indentify the economics environment is getting better or worse.
3. SBA-guaranteed percentage
we consider a higher percantage approve by SBA would indicate the loan is a more secure loan. Adding this feature help to specify the realtion between SMB approval amount and bank approval amount.

In [30]:
def creatRealEst(x):
    x['RealEst'] = [0 if i< 240 else 1 for i in x['Term']]
    return x

In [31]:
def creatSMBpct(x):
    x['SMBApprovelRate'] = x['SBA_Appv']/x['GrAppv']
    return x

In [32]:
def creatGDPGrowth(x):
    x['GDPGrowth'] = [gdp_dict[i] for i in x['ApprovalFY'].values]
    return x

### 2.3 Wrap all feature transform in one function

In [33]:
def data_proessing(x):
    x = drop_cols(x)
    x = re_sorted_bank(x)
    x = resorted_city(x)
    x = sign_clean(x)
    x = clean_newExt(clean_FY(x))
    x = clean_NAICS(x)
    x = type_convert(x)
    x = creatRealEst(x)
    x = creatSMBpct(x)
    x = creatGDPGrowth(x)
    return x  

In [34]:
feature_transformer = FunctionTransformer(data_proessing)

In [35]:
cat_columns = ['City', 'State', 'Bank', 'NAICS', 'ApprovalDate']

In [36]:
con_columns = ['ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'UrbanRural',
               'DisbursementGross', 'GrAppv', 'SBA_Appv', 'RealEst', 'SMBApprovelRate','GDPGrowth']

In [37]:
y = target['MIS_Status'].values

In [38]:
X = loan.copy()

In [48]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=.4)

## 3. Pipeline building & parameter tuning
----
Use 3 models to measure:
1. Random forest calssification
2. Logistic Regression
3. Xgboosting classification

In [40]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [41]:
# Pipeline for converting/processing continuous data
# We don't have too much missing values here, so use most frequent data to replace them
con_pipe = Pipeline([('impute',       SimpleImputer(strategy="most_frequent")),
                     ('scaler', StandardScaler())])

# Pipeline for converting/processing categorical data
cat_pipe = Pipeline([('impute',       SimpleImputer(strategy="most_frequent")),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))])

# Preprocessing pipeline
preprocessing = ColumnTransformer([('categorical', cat_pipe, cat_columns),
                                   ('continuous', con_pipe, con_columns)])

### Using RandomizedSearchCV among 3 models and get the best estimator

In [ ]:
pipe_sc = Pipeline([('feature', feature_transformer),
                    ('preprocessing', preprocessing),
                    ('clf', DummyEstimator())])

# do a RandomizedSearchCV search among the given 3 models, find the best model and best paramater
search_space = [{'clf': [LogisticRegression()], # LogisticRegression
                 'clf__penalty': ['l1', 'l2'], # specify the norm used in the penalization in regulazation
                 'clf__C': np.logspace(0, 5)}, # explore the effect of regularization 
                
                {'clf': [RandomForestClassifier()],  # RandomForest
                 'clf__criterion': ['gini', 'entropy'], # try effect of different loss function
                 'clf__max_depth': [3, 5], # try different depth for tree, contol the number of tree layers
                 'clf__min_samples_leaf': [1, 3], # control min sample number each tree takes
                 'clf__n_estimators': [100, 200]}, # control max trees number generate by model
                
                {'clf': [XGBClassifier()],  # xgboosting
                 # try the loss function to calculate festure importance
                 'clf__importance_type': ["gain", "weight", "total_cover"],
                 'clf__max_depth': [3, 6], # contol the number of tree depth, prevent overfitting
                 'clf__learning_rate': [0.2, 0.3], # control the step of gradient decent to find the optimal
                 'clf__n_estimators': [100, 200]}] # control max trees number generate by model

# Good ol' random search
clf_algos_rand = RandomizedSearchCV(estimator=pipe_sc, 
                                    param_distributions=search_space, 
                                    n_iter=25,
                                    # use balanced_acc because we are positive and negative evenly
                                    scoring = 'balanced_accuracy', 
                                    cv=5, # 5-fold cross validation to get a genralized result
                                    n_jobs=-1)

best_model = clf_algos_rand.fit(X_train, y_train)

# get the paramenter for best model
best_model.best_estimator_.get_params()['clf'] 


In [ ]:
best_model.best_estimator_

In [ ]:
pipe_xgb = Pipeline([('feature', feature_transformer),
                     ('preprocessing', preprocessing),
                     ('best_model', best_model.best_estimator_.get_params()['clf'])])

In [ ]:
pipe_xgb.fit(X_train, y_train)

## 4. Evaluation Metric
----

The data is a binomial imbalanced class, and here, I would like to put evenly weight on both posotove and negative classification, so we use **balanced_accuracy_score** here as a baseline metric, it simply measures generally what the probability that model would put the loan to a correct class. We want balanced weight on both postive class and negative class because the purpose of bank activities should be maximizing the profit by controling the risk instead of only considering the profit.

Also, print out the confustion matrix to check **Specificity**: $\frac{True negative}{conditional negative}$, because I would like to see given a loan is default, what is the probability that the model would make a correct classification.

In [ ]:
y_pred_xgb = pipe_xgb.predict(X_validation)

In [ ]:
print(f'The balanced prediction accuracy is {balanced_accuracy_score(y_validation, y_pred_xgb):.4f}.')

In [ ]:
cfm = confusion_matrix(y_validation, y_pred_xgb)

In [ ]:
print(f'Specificity is {cfm[1][1]/(cfm[0][1]+cfm[1][1]):.2f}')

## 5. Summary

### 5.1 The best fit model with best fit parameter - xgboosting
```python
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='total_cover', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
```

### 5.2 All pipelines steps and all non-default hyperparameters
``` python
Pipeline(steps=[('feature',
                 FunctionTransformer(func=<function data_proessing at 0x7ff9c5fb28b0>)),
                ('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'State', 'Bank',
                                                   'NAICS', 'ApprovalDate']),
                                                 ('continuous',
                                                  Pipeline(step...
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='total_cover',
                               interaction_constraints='', learning_rate=0.2,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=200,
                               n_jobs=16, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])
```

### 5.3 Result interpretation

By doing a random CV search, I found out xgboosting is the best model in perdiction. Xgboosting is a tree-base algorithm using gradient boosting as ensemble method. Bossting is a dynamic ensemble method which updated the tree weight according to the previous step's loss and classification result. Gradient Boosting use Gradient Descent to minimize the loss function in boosting ensemble.

The result from xgb model show below:

|          |  Not default  | default|
|----------|:-------------:|:------:|
|Not default (Actual)|    576085   |  15565  |
|default (Actual) |     20890    |    105194 |

The model successfully calssify 87% of default loan. In other words, given an upcoming loan's information, the model has 90% probabilty to tell if this loan would default correctly. And given a loan is default loan, the model has 87%  probabilty to classify it correctly.

The model can greatly help a bank to decide weather a loan should be approved or not by simplying getting all related input data into this model.

### 5.4. Further to do

1. Find the best split for bank and city. The split in frequency would affect the classification result, if I have more time, I would select the split more carefully by applying gini loss.
2. Find better measurement for economy environment. GDP growth rate is the most obvious measurement for economy, however, it might be too marco. If we would like to track the economy correctly, some other factors like unemployment rate, national debt rate and bank long run interest rate can also be a good measurement.
3. It's an imbalanced dataset, in this notebook I use balanced accurancy and balanced weighted to reduce the effect of imbalanced. Since the whole dataset is large, upsampling is not applied. However, further researh can pay attention to upsampling.